In [ ]:
file = 21 #Change number for file to run

In [ ]:
import numpy as np
import load_data as ld
import random
import matplotlib.pyplot as plt
from scipy import interpolate
from matplotlib.pyplot import figure

In [ ]:
def get_noisy_odometer_path(file, particales, std_move, std_theta):
    
    FR, FL, RR, RL, encoder_time  = ld.get_encoder('data/Encoders'+ str(file))

    width = 733
    data_length = len(encoder_time)
    
    Tic_Dist_Move = (np.pi*254)/360
    right_wheels = np.add(FR,RR)
    left_wheels = np.add(FL,RL)
    right_wheels_dist = (right_wheels*Tic_Dist_Move)/2
    left_wheels_dist  = (left_wheels*Tic_Dist_Move)/2
   
    Odo_Theta = np.zeros((len(np.subtract(right_wheels_dist,left_wheels_dist) / width),particales))
    Move_Direc_X = np.zeros((len(Odo_Theta[:,0]),len(Odo_Theta[0,:])))
    Move_Direc_Y = np.zeros((len(Odo_Theta[:,0]),len(Odo_Theta[0,:])))
    Move_Uncert = np.zeros((len(Odo_Theta[:,0]),len(Odo_Theta[0,:])))
    
    Odo_Theta[:,0] = np.subtract(right_wheels_dist,left_wheels_dist) / width
    
    
    for i in range(len(Odo_Theta[:,0])):
        Odo_Theta[i,1:]=add_noise(Odo_Theta[i,0],particales,std_theta)
             
    for i in range(1, len(Odo_Theta[:,0])):
        for j in range(len(Odo_Theta[0,:])): 
            Odo_Theta[i,j] += Odo_Theta[i-1,j]
            
    Move_Uncert[:,0] = np.add(right_wheels_dist,left_wheels_dist) / 2.0
    
    for i in range(len(Move_Uncert[:,0])):
        Move_Uncert[i,1:]=add_noise(Move_Uncert[i,0],particales,std_move)   
    
    for i in range(len(Odo_Theta[:,0])):
        for j in range(len(Odo_Theta[0,:])):            
            Move_Direc_X[i,j] = Move_Uncert[i,j] * np.cos(Odo_Theta[i,j])
            Move_Direc_Y[i,j] = Move_Uncert[i,j] * np.sin(Odo_Theta[i,j])
    
    X_Odo_Path = Move_Direc_X * 0.001
    Y_Odo_Path = Move_Direc_Y * 0.001
       
    return X_Odo_Path, Y_Odo_Path, Odo_Theta, encoder_time

In [ ]:
def match_time(Lidar, Odo_Time):
    Lidar_Time = []
    Lidar_Index = []
    
    for i in range (len(Lidar)):
        Lidar_Time.append(Lidar[i]['t'])
    
    for i in range((Odo_Time).shape[0]):
        match = np.asarray(Lidar_Time)[(np.abs(Odo_Time[i] - np.asarray(Lidar_Time))).argmin()]
        Lidar_Index.append(int(Lidar_Time.index(match)))
        
    Time_Stamps = np.asarray(Lidar_Index)
    
    return Time_Stamps

In [ ]:
def bresenham2D(sx, sy, ex, ey):
    sx = np.int(round(sx))
    sy = np.int(round(sy))
    ex = np.int(round(ex))
    ey = np.int(round(ey))
    dx = abs(ex-sx)
    dy = abs(ey-sy)
    steep = abs(dy)>abs(dx)
    if steep:
        dx,dy = dy,dx # swap 
    if dy == 0:
        q = np.zeros((dx+1,1))
    else:
        q = np.append(0,np.greater_equal(np.diff(np.mod(np.arange( np.floor(dx/2), -dy*dx+np.floor(dx/2)-1,-dy),dx)),0))
    if steep:
        if sy <= ey:
            y = np.arange(sy,ey+1)
        else:
            y = np.arange(sy,ey-1,-1)
        if sx <= ex:
            x = sx + np.cumsum(q)
        else:
            x = sx - np.cumsum(q)
    else:
        if sx <= ex:
            x = np.arange(sx,ex+1)
        else:
            x = np.arange(sx,ex-1,-1)
        if sy <= ey:
            y = sy + np.cumsum(q)
        else:
            y = sy - np.cumsum(q)
    return np.vstack((x,y)).astype(np.int)

In [ ]:
def polar2cart(scan, angle, Odo_Theta):
    X=(scan * np.cos(angle))*np.cos(Odo_Theta)-(scan * np.sin(angle))*np.sin(Odo_Theta)
    Y=(scan * np.cos(angle))*np.sin(Odo_Theta)+(scan * np.sin(angle))*np.cos(Odo_Theta)
    return X, Y

In [ ]:
def add_noise(x,p,std):
    return np.random.normal(x, std, p-1)

In [ ]:
def create_map(Conv_Lidar ,X_Odo_Path,Y_Odo_Path,Size,MAP,i):

    ScaleX = np.round(Conv_Lidar[i]['X']+X_Odo_Path,2)
    ScaleY = np.round(Conv_Lidar[i]['Y']+Y_Odo_Path,2)
    X_Pos = (ScaleX*10+Size/2).astype(np.int)
    Y_Pos = (ScaleY*10+Size/2).astype(np.int)

    for j in range(len(X_Pos)):       
        Stack = bresenham2D((X_Odo_Path*10+Size/2),(Y_Odo_Path*10+Size/2),X_Pos[j][0],Y_Pos[j][0])
        Stack_Miss = Stack[:, 0:(Stack.shape[1]-2)]
        Stack_Hit = Stack[:, (Stack.shape[1]-1)]
        MissY = Stack[0,:-1]
        MissX = Stack[1,:-1]
        HitY = Stack[0,-1:]
        HitX = Stack[1,-1:]

        if((Stack_Miss.size > 0) and (Stack_Hit.size > 0)):
            MAP[MissY,MissX] -= 0.1
            MAP[HitY,HitX] += 0.5
            
    return MAP

In [ ]:
def corr_calc(Conv_Lidar,Particles ,MAP, Size,i):
    
    X_Particle = Particles[0]
    Y_Particle = Particles[1]
    
    ScaleX = np.round(Conv_Lidar[i]['X']+X_Particle,2)
    ScaleY = np.round(Conv_Lidar[i]['Y']+Y_Particle,2)
    X_Pos = (ScaleX*10+Size/2).astype(np.int)
    Y_Pos = (ScaleY*10+Size/2).astype(np.int)
    Hit_Count=0
    for j in range(len(X_Pos)):
        if ((MAP[X_Pos[j],Y_Pos[j]][0]) > 0):
            Hit_Count += 1
                
    return Hit_Count

In [ ]:
def particle_resample(Weights, Particles):
    Weights = Weights * 10000
    Weight_Range = np.zeros(len(Weights))
    Weight_Range[0] = Weights[0]
    New_Particles = np.zeros((len(Particles),2))
      
    for i in range (1, len(Weights)):        
        Weight_Range[i] =  Weight_Range[i-1] + Weights[i]
        
    Wheight_Max = round(Weight_Range[(len(Weight_Range)-1)])
    
    Picks = np.random.randint(Wheight_Max, size = len(Particles[:,0]))
    
    for i in range (len(Particles)):
        result = np.where(Weight_Range[:] > Picks[i])
        New_Particles[i,:] = Particles[(result[0][0]),:]
    
    return New_Particles

In [ ]:
def particle_resample_test(Weights):
    Top = np.square(np.sum(Weights))
    Bot = np.sum(np.square(Weights))
    N_Eff = Top/Bot
    
    return N_Eff 

In [ ]:
Particle_Count = 60
std_theta = 0.004
std_move = 0.0004
Particles = np.zeros((Particle_Count,2))
Weight_List = np.zeros((Particle_Count))
Weight_List_temp = np.zeros((Particle_Count))
Weight_List[:] = 1

X_Odo_Path, Y_Odo_Path, Odo_Theta, Odo_Time = get_noisy_odometer_path(file,Particle_Count,std_move, std_theta)

Size = 900
MAP = np.zeros((Size,Size))

Lidar = ld.get_lidar('data/Hokuyo'+ str(file))
Time_Stamps  = match_time(Lidar, Odo_Time)    
Conv_Lidar = np.take(Lidar,Time_Stamps)

for i in range (len(Time_Stamps)):
    Lidar_Scans = np.reshape(Conv_Lidar[i]['scan'],(len(Conv_Lidar[i]['scan']),1))
    Lidar_Angles = np.reshape(Conv_Lidar[i]['angle'],(len(Conv_Lidar[i]['angle']),1))

    threshold_indices = Lidar_Scans > 10
    Lidar_Scans[threshold_indices] = 10

    Conv_Lidar[i]['X'], Conv_Lidar[i]['Y'] = polar2cart(Lidar_Scans,Lidar_Angles,Odo_Theta[i])
    
MAP = create_map(Conv_Lidar,X_Odo_Path[0][0],Y_Odo_Path[0][0],Size, MAP, 0)

Path = np.zeros((len(Time_Stamps),2))

Particles[:,0] = X_Odo_Path[0,:]
Particles[:,1] = Y_Odo_Path[0,:]
Path[0,0] = Particles[0,0]
Path[0,1] = Particles[0,1]


for i in range(1 ,len(Time_Stamps)):
    
    Particles[:,0] += X_Odo_Path[i,:]
    Particles[:,1] += Y_Odo_Path[i,:]
        
    if ((Particles[0,0] != 0) or (Particles[0,1] != 0)):

        print(i)
        for j in range(len(Particles)):
            Weight_List_temp[j] = corr_calc(Conv_Lidar,Particles[j,:],MAP, Size,i)
            
        High_temp = np.argmax(Weight_List_temp)
        Weight_List_temp = Weight_List_temp/(Weight_List_temp[High_temp])
        
        Weight_List = Weight_List * Weight_List_temp
        High = np.argmax(Weight_List)

        print(High)
        Path[i,0] = Particles[High,0]
        Path[i,1] = Particles[High,1]
        MAP=create_map(Conv_Lidar,Particles[High,0],Particles[High,1],Size,MAP,i)

        MAP = np.where(MAP < -20, -20, MAP)
        MAP = np.where(MAP > 20, 20, MAP)
        
        N_Eff = particle_resample_test(Weight_List)
        print(N_Eff)


        if (N_Eff < (Particle_Count/1.5)):
            print(Weight_List)
            Particles = particle_resample(Weight_List, Particles)
            Weight_List[:] = 1
            print('resample')
        
MAP = np.where(MAP < 0, 0, MAP)
MAP = np.where(MAP > 20, 20, MAP)

convert_gray=interpolate.interp1d([-20,20],[0,255])
gray_map = convert_gray(MAP.T)

figure(figsize=(10, 10))
plt.title('Occupancy Grid and robot path for '+ str(file))
plt.xlabel('X-direction (m x 10)')
plt.ylabel('Y-direction (m x 10)')
plt.scatter(np.round(Path[:,0],2)*10+Size/2,np.round(Path[:,1],2)*10+Size/2,s=0.1)
plt.imshow(gray_map,cmap='gray')
plt.show() 


In [ ]:
figure(figsize=(10, 10))
plt.title('Occupancy Grid and robot path for '+ str(file))
plt.xlabel('X-direction (m x 10)')
plt.ylabel('Y-direction (m x 10)')
plt.scatter(np.round(Path[:,0],2)*10+Size/2,np.round(Path[:,1],2)*10+Size/2,s=0.1)
plt.imshow(gray_map,cmap='gray')
plt.show() 